In [1]:
import ocsmesh
import geopandas as gpd

In [2]:
## Inputs:
path = r"C:\Users\Felicio.Cassalho\Work/Modeling/NHC/"
points = gpd.read_file(path+"/sample.shp")
stofs_mesh = ocsmesh.Mesh.open(r"C:\Users\Felicio.Cassalho\Work\Modeling\OCSMesh_Paper\STOFS_2D_RUN\R05/hgrid.gr3", crs=4326)

In [3]:
## First we clip the STOFS mesh using the points. 
## I added a 0.011 deg buffer to the points, I am grabbing all nodes that fall within that buffer (no adjacent laryers)
## This seems to be working, feel free to change the buffer size and adj layers.

clip = ocsmesh.utils.clip_mesh_by_shape(stofs_mesh.msh_t, points.union_all().buffer(0.011), adjacent_layers=0)
## if you want to see the clipped mesh uncomment below:
#ocsmesh.Mesh(clip).write(path+"/clip.2dm", format='2dm', overwrite=True)

In [4]:
## Converting the mesh clip to polygon: 
clip_poly = ocsmesh.utils.get_mesh_polygons(clip)
clip_gdf = gpd.GeoDataFrame(geometry =
                              gpd.GeoSeries(clip_poly),
                                crs=4326).explode()
# you can save the geodataframe as a shapefile and open on QGis if needed

In [5]:
## This is the main line of code. 
## This uses the Triangle library (very light weight and efficient) to create a mesh for the polygon above (clips)
## note the opts:
## q means exact (will preserve the boundaries)
## q30 will ensure that the internal angles of the mesh will be at least 30 deg.
## a.0000002 is the area of the max area triangles (this seems to be +- 50 m), change accordingly
clip_mesh = ocsmesh.utils.triangulate_polygon(clip_gdf,opts='pq30a.0000002')

## save the mesh if you want to visualize it:
## ocsmesh.Mesh(clip_mesh).write(path+"/clip_mesh.2dm", format='2dm', overwrite=True)

In [6]:
stofs_mesh

In [ ]:
## Now we just merge the clip_mesh to the stofs mesh:
final_mesh = ocsmesh.utils.merge_overlapping_meshes([stofs_mesh.msh_t,clip_mesh],adjacent_layers=2,buffer_size = 0.0005)

In [ ]:
ocsmesh.Mesh(final_mesh).write(path+"/final_mesh.2dm", format='2dm', overwrite=True)